In [7]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

### Keras setup

In [8]:
# convert to one-hot encoding
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler

### Data loading

In [9]:
train_file = "data/train.csv"
test_file = "data/test.csv"
output_file = "submission.csv"

### Split training data as training set and validation set

In [ ]:
raw_data = np.loadtxt(train_file, skiprows=1)